## LCM Function

In [ ]:
import json
import random
import constraint
from techniques.PaDMTP import padmtp_algo
from techniques.RandomTesting import random_testing
from testcodes.lcm import compute_lcm as lcm
from techniques.AdaptiveRandomTesting import adaptive_random_testing

### PaDMTP

In [ ]:
problem = constraint.Problem()

problem.addVariable('x', range(1, 16))
problem.addVariable('y', range(1, 16))

problem.addConstraint(lcm, ['x', 'y'])

solutions = problem.getSolutions()

length = len(solutions)

In [ ]:
# Cleaning Dataset where values of 'x' or 'y' are 0(zero)

cleaned_solutions = []

for index, solution in enumerate(solutions):
    if solution['x'] == 0 or solution['y'] == 0:
        continue
    else:
        cleaned_solutions.append(solutions[index])
        
lcm_length = len(cleaned_solutions)

print("\nPossible Solutions Dataset for LCM Code: \n(x,y) ∈ {", end="")
for index, solution in enumerate(cleaned_solutions):
    if index == lcm_length - 1:
        print("({},{})".format(solution['x'], solution['y']), end="")
    else:
        print("({},{}),".format(solution['x'], solution['y']), end="")
print("}\n")

In [ ]:
print(f"LCM - PaDMTP - Test Cases: {lcm_length}")

In [ ]:
mutPyObj = 'mut.py --target testcodes.lcm --unit-test mutations.test_lcm_padmt -m'

padmtp_rslt = padmtp_algo(lcm, cleaned_solutions, mutPyObj)

In [ ]:
padmtp_rslt['PaDMTP_Overhead']

In [ ]:
padmtp_rslt['PaDMTP_DataFrame']

In [ ]:
PaDMTP_MT_Scores = []

for index, row in padmtp_rslt['PaDMTP_DataFrame'].iterrows():
    val = row['MT Score']
    val = val.replace("%", "")
    val = float(val) / 100
    val = round(val, 3)
    
    PaDMTP_MT_Scores.append(val)

### Random Testing (RT)

In [ ]:
# Generating random inputs
rt_test_cases = []
for i in range(lcm_length):
    x = random.randint(1, 101)
    y = random.randint(1, 101)
    
    rt_test_cases.append((x, y))

In [ ]:
print(f"LCM - Random Testing - Test Cases: {len(rt_test_cases)}")

In [ ]:
# Saving RT Test Cases to JSON File

jsonString = json.dumps(rt_test_cases)
jsonFile = open("./temp/RT_Test_Cases.json", 'w')
jsonFile.write(jsonString)
jsonFile.close()

In [ ]:
pytest_cmd = 'pytest techniques/rt_lcm.py'
mutpy_cmd = 'mut.py --target testcodes.lcm --unit-test mutations.test_lcm_rt -m'

rt_rslt = random_testing(pytest_cmd, mutpy_cmd, rt_test_cases)

In [ ]:
rt_rslt['RT_Overhead']

In [ ]:
rt_rslt['RT_DataFrame']

In [ ]:
RT_MT_Scores = []

for index, row in rt_rslt['RT_DataFrame'].iterrows():
    val = row['MT Score']
    val = val.replace("%", "")
    val = float(val) / 100
    val = round(val, 3)
    
    RT_MT_Scores.append(val)

### Adaptive Random Testing (ART)

In [ ]:
# Generating random inputs
art_test_cases = []
for i in range(lcm_length):
    x = random.randint(1, 101)
    y = random.randint(1, 101)
    
    art_test_cases.append([x, y])

In [ ]:
print(f"LCM - Adaptive Random Testing - Test Cases: {len(art_test_cases)}")

In [ ]:
mutpy_cmd = 'mut.py --target testcodes.lcm --unit-test mutations.test_lcm_art -m'

art_rslt = adaptive_random_testing(lcm, mutpy_cmd, art_test_cases, 'LCM')

In [ ]:
art_rslt['ART_Overhead']

In [ ]:
art_rslt['ART_DataFrame']

In [ ]:
ART_MT_Scores = []

for index, row in art_rslt['ART_DataFrame'].iterrows():
    val = row['MT Score']
    val = val.replace("%", "")
    val = float(val) / 100
    val = round(val, 3)
    
    ART_MT_Scores.append(val)

### Pairwise T-Test

In [ ]:
import scipy.stats as stats
import numpy as np
from tabulate import tabulate

# Define the data
PaDMTP = np.array(PaDMTP_MT_Scores) 
RT = np.array(RT_MT_Scores)
ART = np.array(ART_MT_Scores)

# Calculate the mean
mean_padmtp = np.mean(PaDMTP)
mean_rt = np.mean(RT)
mean_art = np.mean(ART)

# Calculate the variance
var_padmtp = np.var(PaDMTP, ddof=1)
var_rt = np.var(RT, ddof=1)
var_art = np.var(ART, ddof=1)

# Get the number of observations
n_padmtp = len(PaDMTP)
n_rt = len(RT)
n_art = len(ART)

# Calculate the degrees of freedom
df_padmtp = n_padmtp - 1
df_rt = n_rt - 1
df_art = n_art - 1

# Perform t-tests
t_stat_rt, _ = stats.ttest_1samp(RT, popmean=mean_padmtp)
t_stat_art, _ = stats.ttest_1samp(ART, popmean=mean_padmtp)

# Create tables
table1 = [
    ["Test", "Mean", "Variance", "Observations", "df", "t Stat"],
    ["PaDMTP vs RT", mean_rt, var_rt, n_rt, df_rt, t_stat_rt],
    ["PaDMTP vs ART", mean_art, var_art, n_art, df_art, t_stat_art]
]

print("<==== LCM Code ====>")

# Print tables
print("Table 1:")
print(tabulate(table1, headers="firstrow", tablefmt="grid"))
# Perform one-sample t-tests
t_stat_rt, p_value_rt = stats.ttest_1samp(RT, popmean=np.mean(PaDMTP))
t_stat_art, p_value_art = stats.ttest_1samp(ART, popmean=np.mean(PaDMTP))

# Set the significance level (alpha)
alpha = 0.05

# Calculate the one-tail p-values and critical values
p_one_tail_rt = stats.t.cdf(t_stat_rt, df=len(RT)-1)
p_one_tail_art = stats.t.cdf(t_stat_art, df=len(ART)-1)
t_crit_one_tail_rt = stats.t.ppf(1-alpha, df=len(RT)-1)
t_crit_one_tail_art = stats.t.ppf(1-alpha, df=len(ART)-1)

# Calculate the two-tail p-values and critical values
p_two_tail_rt = 2 * (1 - stats.t.cdf(abs(t_stat_rt), df=len(RT)-1))
p_two_tail_art = 2 * (1 - stats.t.cdf(abs(t_stat_art), df=len(ART)-1))
t_crit_two_tail_rt = stats.t.ppf(1-alpha/2, df=len(RT)-1)
t_crit_two_tail_art = stats.t.ppf(1-alpha/2, df=len(ART)-1)
print("\n\n")
# Create table
table2 = [
    ["Test", "P(T<=t) one-tail", "t Critical one-tail", "P(T<=t) two-tail", "t Critical two-tail"],
    ["PaDMTP vs RT", p_one_tail_rt, t_crit_one_tail_rt, p_two_tail_rt, t_crit_two_tail_rt],
    ["PaDMTP vs ART", p_one_tail_art, t_crit_one_tail_art, p_two_tail_art, t_crit_two_tail_art]
]

# Print table
print("Table 2:")
print(tabulate(table2, headers="firstrow", tablefmt="grid"))
print("\n\n")
# Perform one-sample t-tests
t_stat_rt, p_value_rt = stats.ttest_1samp(RT, popmean=0.727)
t_stat_art, p_value_art = stats.ttest_1samp(ART, popmean=0.727)
print("Table 3:")
# Print the results
print("RT vs PaDMTP:")
print("t Statistic:", t_stat_rt)
print("p-value:", p_value_rt)
print("\nART vs PaDMTP:")
print("t Statistic:", t_stat_art)
print("p-value:", p_value_art)